In [1]:
#  cd OneDrive\INSPER\PADS\TCC\FINAL\TABELA_ERRO_PREVISÃO\

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import math

In [4]:
import itertools

from functools import reduce

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

In [6]:
#import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive,interactive_output
from ipywidgets import Layout, Button, Label
from ipywidgets import GridspecLayout,HBox, VBox, Box

In [7]:
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

In [8]:
regressao=['knn','gb','mlr','rf']
regressao_mlr=['knn','gb','rf']
regressao=['knn','gb','rf']

In [9]:
url='https://raw.githubusercontent.com/LeonardoGiovanelli/TCC_LCA/main/erro2/df_erro_EQM_indici_data_regressao_tsw_5.csv'
erro_adjested_tidy=pd.read_csv(url)
erro_adjested_tidy=erro_adjested_tidy.set_index(['Indice','Nd','Regressao','tsw'])
erro_adjested_tidy=erro_adjested_tidy.round(2)
erro_adjested_tidy.rename(columns={'Mes':"Mês"},inplace=True)

In [10]:
indices=list(erro_adjested_tidy.index.levels[0])

In [11]:
url='https://raw.githubusercontent.com/LeonardoGiovanelli/TCC_LCA/main/previsao2/df_PREVISAO_indici_data_regressao_tsw.csv'
previsao=pd.read_csv(url).iloc[:,1:]
previsao=previsao.round(2)
previsao=previsao.set_index(['Indice','Nd','Regressao','tsw'])
previsao=previsao.rename(columns={"Eroo": "Erro"})
#previsao.head()

In [12]:
def df_color(df,cmap,low,high): 
    dff=df.style.background_gradient(cmap=cmap, low=low, high=high)
    return(dff)

In [13]:
from itertools import compress, product

def combinations(items):
    return ( set(compress(items,mask)) for mask in product(*[[0,1]]*len(items)) )

In [14]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / (w.sum())

In [15]:
def add_data(r):
    r['Dia']="1"
    r["Ano"]=r["Ano"].apply(lambda x: math.trunc(x))
    r["Mês"]=r["Mês"].apply(lambda x: math.trunc(x))
    r['data']=pd.to_datetime(r["Ano"].astype(str) + "/" + r["Mês"].astype(str) + "/" + r["Dia"])
    return(r)

In [16]:
# #########ERRO

In [17]:
from sklearn.metrics import r2_score
import math

def mean_erro(indice_nome,reg):

    p=erro_adjested_tidy.loc[indice_nome,:,reg,:].groupby(['Ano','Mês']).\
    apply(lambda x: np.average(x.Previsto, weights=1/(x['Real-Previsto']+.0002)))
    p=pd.DataFrame(p)
    
    erro=erro_adjested_tidy.loc[indice_nome,:,reg,:].groupby(['Ano','Mês']).\
    apply(lambda x: np.average(x.Erro, weights=1/(x['Real-Previsto']+.0002)))
    erro=pd.DataFrame(erro).reset_index().iloc[:,-1]
        
    r=erro_adjested_tidy.loc[indice_nome,0,'knn',0]
    r=r['Real']
    r=r.reset_index()

    p=pd.concat([p.reset_index(),r],axis=1,ignore_index=True)
    p=p[[0,1,7,2]]

    p.columns=['Ano','Mes','Real','Previsto']
    
    p['Erro']=erro
    p['Real-Previsto']=p['Real']-p['Previsto']
    p['Eqm(%)']=100*((p['Real-Previsto']/p['Real'])**2)**.5
  
    return(p)

In [18]:
def mean_erro_index(indice_nome):
    
    regressao=['knn','gb','rf']
    
    reg_comb=list(combinations(regressao))[1:]
    reg_comb=list(reg_comb)

    t=[mean_erro(indice_nome,r) for r in reg_comb]

    t_rp_mean=[np.mean(x['Real-Previsto']**2)**0.5 for x in t]
    t_rp_std=[np.std((x['Real-Previsto'])) for x in t]

    t_eqm_mean=[np.mean(x['Eqm(%)']**2)**.5 for x in t]
    t_eqm_std=[np.std(abs(x['Eqm(%)'])) for x in t]

    t_r2=[r2_score(x['Real'],x['Previsto']) for x in t]

    t_pd=pd.DataFrame([reg_comb,t_rp_mean,t_rp_std,t_eqm_mean,t_eqm_std,t_r2]).transpose()
    t_pd.columns=['reg','rp_mean','rp_std','eqm_mean(%)','eqm_std(%)','r2']

    #var='eqm_mean(%)'
    var='r2'
    
    if var=='eqm_mean(%)':
        t_summary=t_pd.sort_values(by=[var],ascending=True)
    if var=='r2':
        t_summary=t_pd.sort_values(by=[var],ascending=False)
    
    best_index=t_summary.index[0]
    t_best=t[best_index]
    
    return([t_summary,t_best.rename(columns={'Mes':'Mês'})])

In [19]:
erro_merged=[mean_erro_index(i) for i in indices]

In [20]:
erro_comp=pd.concat([x[0].iloc[0,-3:] for x in erro_merged],axis=1).reset_index()
erro_comp.iloc[0:2,:]=erro_comp.iloc[0:2,:].round(0)
erro_comp=erro_comp.set_index('index')
erro_comp.columns=indices
erro_comp.index.names = ['']
erro_comp=erro_comp.transpose().sort_values('r2',ascending=False)

In [21]:
teste=erro_adjested_tidy

teste_0=teste[np.in1d(teste.index.get_level_values(1), [0])]
teste_0=teste_0.groupby(['Indice','Regressao']).mean()
teste_0f=teste_0.reset_index().pivot(index='Indice', columns='Regressao', values='Real-Previsto(%)').round(0)


teste_1=teste[np.in1d(teste.index.get_level_values(1), [1])]
teste_1=teste_1.groupby(['Indice','Regressao']).mean()
teste_1f=teste_1.reset_index().pivot(index='Indice', columns='Regressao', values='Real-Previsto(%)').round(0)

In [22]:
####### previsao

In [23]:
def mean_previsao(indice_nome):
    
    indices=list(previsao.index.levels[0])
    index_number=indices.index(indice_nome)
    
    reg=erro_merged[index_number][0].iloc[0,0]
    #############
    
    p=previsao.loc[indice_nome,:,reg,:].groupby(['Ano','Mês']).\
    apply(lambda x: np.average(x.Previsão, weights=1/(x['EQM(%)']+.000002)))
    p=pd.DataFrame(p)

    erro=previsao.loc[indice_nome,:,reg,:].groupby(['Ano','Mês']).\
    apply(lambda x: np.average(x.Erro, weights=1/(x['EQM(%)']+.000002)))
    erro=pd.DataFrame(erro)
    
    eqm=previsao.loc[indice_nome,:,reg,:].groupby(['Ano','Mês'])['EQM(%)'].mean()
    df=pd.concat([p,erro,eqm],axis=1,ignore_index=True)

    df.columns=['Previsão','Erro','EQM(%)']
    return(add_data(df.reset_index()))

In [24]:
previsao_merged=[mean_previsao(i) for i in indices]

In [25]:
def plot_prev(index):
 
    colors=["black","blue"]
   
    indices=list(erro_adjested_tidy.index.levels[0])
    index_number=indices.index(index)
    
############# data   
    r=previsao_merged[index_number].round(2)              

    min=np.min(r['Previsão']-r['Erro'])
    max=np.max(r['Previsão']+r['Erro'])
########################################    


In [26]:
def plot_erro(index):
 
 ############ get data 
    colors=["black","blue"]
   
    indices=list(erro_adjested_tidy.index.levels[0])
    index_number=indices.index(index)
    r=add_data(erro_merged[index_number][1]).round(2)              

    min=np.min(np.min(r[['Real','Previsto']]))
    max=np.max(np.max(r[['Real','Previsto']]))
    
########################################    
    fig = go.Figure() 
    
    fig.add_trace(go.Scatter(
    line=dict(color=colors[0]),
    x=r['data'],
    y=r['Previsto'],
     error_y=dict(
   type='data',
   symmetric=True,
   array=r['Erro']),
     name="Previsto"))

    fig.add_trace(go.Scatter(
        line=dict(color=colors[1]),
        x=r['data'],
        y=r['Real'],
       name="Real"))

    fig.update_xaxes(dtick="M1",    tickformat="%b\n%Y")
    fig.update_layout(title={'text':index,'x':0.5,'xanchor': 'center'})
    fig.show()

In [27]:
def plot_prev_erro_bar(index):
    
    indices=list(erro_adjested_tidy.index.levels[0])
    n=indices.index(index)
################    
    e=erro_merged[n][1]
    #e=e[e.Ano>=2020]
    e=add_data(e)
    e=e.iloc[:,[-1,2,3,4]]
    e=e.rename(columns={"Previsto":"Previsão"})
##############
    p=previsao_merged[n]
    p=add_data(p)
    p=p.iloc[:,[-1,2,3]]
###############    
    ep=pd.concat([e,p],axis=0).reset_index()
###############    
    ep_m=pd.melt(ep, id_vars=['data'], value_vars=['Real','Previsão']).dropna().reset_index()
    

    l=[[0]*len(ep_m[ep_m.variable=="Real"]),list(e['Erro'].append(p['Erro']).reset_index().iloc[:,1])]
    l = list(itertools.chain(*l))
    erro=l
    ep_m['Erro']=erro
    ep_m=ep_m.round(2)
    
###############################    
    fig = px.bar(ep_m, 
             x="data", y="value", 
             color="variable",
             error_y="Erro",
            barmode='group',
            color_discrete_sequence=["blue", "black"])

    min=np.min(ep_m['value']-ep_m['Erro'])*0.99
    max=np.max(ep_m['value']+ep_m['Erro'])*1.01
    fig.update_yaxes(range=[min,max])

    fig.show()

In [28]:
def table_index(index):
    
    indices=list(erro_adjested_tidy.index.levels[0])
    index_number=indices.index(index)

    reg=erro_merged[index_number][0].iloc[0,0]

    r_boxplot=previsao.loc[indices[index_number],:,reg,:]
    r_boxplot=add_data(r_boxplot)
    r_boxplot=r_boxplot.reset_index()
    r_boxplot=r_boxplot.iloc[:,:-3]
    
    r_boxplot['Erro(%)']=100*r_boxplot['Erro']/r_boxplot['Previsão']
    r_boxplot=r_boxplot.iloc[:,4:].groupby(['Ano','Mês']).mean().round(2)
    
    display(r_boxplot)
   #return(r_boxplot)


In [29]:
def plot_prev_box(index):
    
    indices=list(erro_adjested_tidy.index.levels[0])
    index_number=indices.index(index)
    
    reg=erro_merged[index_number][0].iloc[0,0]
    
    r_boxplot=previsao.loc[index,:,reg,:]
    r_boxplot=add_data(r_boxplot)
    
    r_boxplot
    
    fig = px.box(r_boxplot, 
                 x = "data",
                 y="Previsão",
                 points=False,
                  title="Box plot da previsão"
                )
    
    fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
    
    fig.show()
    

In [30]:
def plot_erro_box(index):
    
    indices=list(erro_adjested_tidy.index.levels[0])
    index_number=indices.index(index)
    
    reg=erro_merged[index_number][0].iloc[0,0]
    
    r_boxplot=previsao.loc[index,:,reg,:]
    r_boxplot=add_data(r_boxplot)
    
    r_boxplot
    
    fig = px.box(r_boxplot, 
                 x = "data",
                 y="Erro",
                 points=False,
                  title="Box plot do erro"
                )
    
    fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
    
    fig.show()

In [ ]:
#table_index('IE_E')

In [ ]:
####ipywidgets

In [ ]:
indices=ipywidgets.Dropdown(options=indices,  value=indices[0],  
                            description='Índice: ',
                           layout={'width': 'max-content'})

nd=ipywidgets.Dropdown(options=[0,1],  value=0, 
                       description='nd:',
                      layout={'width': 'max-content'})

regressao=ipywidgets.Dropdown(options=regressao,  value=regressao[0],  
                              description='Regressão:',
                             layout={'width': 'max-content'})

tsw=ipywidgets.Dropdown(options=[0,1],  value=0,  
                        description='tsw:',
                       layout={'width': 'max-content'})


info=ipywidgets.HBox([indices,nd,regressao,tsw])

In [ ]:
from IPython.display import display_html 
    
out1 = ipywidgets.Output()
out2 = ipywidgets.Output()
out3= ipywidgets.Output()

######## OUT1##################
with out1:
         display(erro_comp)
###### OUT 2#####################    

#g2=ipywidgets.interactive_output(plot_erro,  
    #{'index':indices})

g2_2=ipywidgets.interactive_output(plot_prev_erro_bar,  
    {'index':indices})


out2=ipywidgets.VBox([ipywidgets.HBox([indices]),g2_2])

####   OUT3   #####################    
#g3_1=ipywidgets.interactive_output(plot_prev,  
  #  {'index':indices})

g3_11=ipywidgets.interactive_output(table_index,  
    {'index':indices})


g3_2=ipywidgets.interactive_output(plot_prev_box,  
    {'index':indices})



g3_3=ipywidgets.interactive_output(plot_erro_box,  
    {'index':indices})



out3=ipywidgets.VBox([
    ipywidgets.HBox([indices,g3_11]),
    ipywidgets.HBox([g3_2,g3_3]),
    ])

    
    
##############################    
tab = ipywidgets.Tab(children = [out1, out2,out3])
tab.set_title(0, 'Indíces')
tab.set_title(1, 'Previsto X Real')
tab.set_title(2, 'Previsão detalhada')

display(tab)